In [ ]:
with open('data') as f:
    source = f.read()

In [54]:
query = 'data'

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline
import torch

# model_id = 'data'
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(model_id)

# tokenizer.save_pretrained("data")
# model.save_pretrained("data")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("data")
model = AutoModelForCausalLM.from_pretrained("data")

In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
local_llm = HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0.1, 'max_length':60})

In [ ]:
# query = 'data'

# inputs= tokenizer.encode(query, return_tensors='pt')
# outputs= model.generate(inputs, max_length=200, num_return_sequences=1, temperature=0.2)

# for i, output in enumerate(outputs):
#     print(f"{i}: {tokenizer.decode(output)}")

In [34]:
from langchain import PromptTemplate, LLMChain

template = '''
Question: {question}
Answer: 
'''

document_prompt = PromptTemplate(template=template, input_variables=['question'])
llm_chain = LLMChain(prompt=document_prompt, llm=local_llm)

In [ ]:
result = llm_chain.invoke(query)
print(result['text'])

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=0)
docs = text_splitter.split_text(source)

embeddings = HuggingFaceEmbeddings()
db = FAISS.from_texts(docs, embeddings)

In [73]:
docs = db.similarity_search(query)
# print(content[0].page_content)

In [ ]:
from langchain.chains.question_answering import load_qa_chain

qa_chain = load_qa_chain(local_llm, chain_type='stuff')
print(qa_chain.run(input_documents=docs, question=query).split('\n\n')[1:])